In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import os
from geopy.distance import geodesic
import sys


df = pd.read_csv(os.path.join('data','Zomato.csv'))
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12/2/2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,4/3/2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,9:20,9:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [2]:
df = df.drop(["ID","Delivery_person_ID","Delivery_person_Age",'Time_Orderd','Time_Order_picked',"Delivery_person_Ratings","Order_Date","Festival"],axis=1)

In [3]:
df.loc[df['Weather_conditions'].isna(), 'Weather_conditions'] = 'Sunny'
df.loc[df['Road_traffic_density'].isna(), 'Road_traffic_density'] = 'Low'
df.loc[df['City'].isna(), 'City'] = 'Semi-Urban'
df.loc[df['multiple_deliveries'].isna(), 'multiple_deliveries'] = 3.0
conditon = df['Restaurant_latitude']== 0
df= df[~conditon]

In [4]:
df['Distance'] = df.apply(lambda row: round(geodesic((row['Restaurant_latitude'], row['Restaurant_longitude']), (row['Delivery_location_latitude'], row['Delivery_location_longitude'])).km,2), axis=1,)

In [5]:
condition = df['Distance'] > 20
df.drop(df[condition].index, inplace=True)

In [7]:
df.isna().sum()

Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
City                           0
Time_taken (min)               0
Distance                       0
dtype: int64

In [25]:
X = df.drop(labels=['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude','Time_taken (min)'], axis=1)

In [9]:
Y = df[['Time_taken (min)']]

In [60]:
df.head(2)

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,City,Time_taken (min),Distance
0,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,Metropolitian,46,10.27
1,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,Metropolitian,23,6.23


In [62]:
categorical_cols = ['Weather_conditions','Road_traffic_density','Type_of_order','Type_of_vehicle','City']

In [63]:
numerical_cols = ['Vehicle_condition','multiple_deliveries','Distance']

In [64]:
road_traffic = ['Jam', 'High', 'Medium', 'Low']
weather_condition = ['Fog','Stormy','Sandstorms','Windy','Cloudy','Sunny']
Type_of_order = ['Snack', 'Meal', 'Drinks', 'Buffet']
type_of_vehicle = ['motorcycle', 'scooter', 'electric_scooter', 'bicycle']
city = ['Metropolitian', 'Urban', 'Semi-Urban']

In [65]:
from sklearn.impute import SimpleImputer  #handling missing values
from sklearn.preprocessing import StandardScaler # handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #it will assign 1,2,3,4 to our categorial 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [66]:
df.head(1)

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,City,Time_taken (min),Distance
0,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,Metropolitian,46,10.27


In [67]:
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[weather_condition, road_traffic, Type_of_order, type_of_vehicle, city])),
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])
print(preprocessor)

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Vehicle_condition', 'multiple_deliveries',
                                  'Distance']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fog',
                                                                              'Stormy',
                                                                              'Sandstorms',
                                                               

In [68]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30, random_state=30)

In [69]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

In [70]:
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor()
}

In [71]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [72]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [74]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [78]:
models = {
    "LinearRegression": LinearRegression(n_jobs=-1),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "RandomForest": RandomForestRegressor()
}
def train_model(model, model_name, x_train= X_train, x_test = X_test, y_train = y_train, y_test = y_test):
    print("#"*50)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(model_name,":")
    print("r2_score", r2_score(y_test, y_pred))
    print("Score", ":", mean_squared_error(y_pred, y_test))
    print("Mean score", ":", np.sqrt(mean_squared_error(y_pred, y_test)))
for model_name, model in models.items():
    train_model(model, model_name)

##################################################
LinearRegression :
r2_score 0.3761770786515737
Score : 54.86351179175195
Mean score : 7.4069907379280515
##################################################
Ridge :
r2_score 0.3761740634348073
Score : 54.863776971781334
Mean score : 7.4070086385653235
##################################################
Lasso :
r2_score 0.31174636963593605
Score : 60.53001560695244
Mean score : 7.780103830088159
##################################################
ElasticNet :
r2_score 0.30578979610502
Score : 61.05387988152167
Mean score : 7.813698220530511
##################################################


/tmp/ipykernel_585/2228399505.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForest :
r2_score 0.5370645931201778
Score : 40.71389698100691
Mean score : 6.3807442341005105
